## Load up the environment variable needed for module imports

In [1]:
%env MIRA_REST_URL=http://34.230.33.149:8771

env: MIRA_REST_URL=http://34.230.33.149:8771


## Import modules

In [2]:
from mira.sources import biomodels
from mira.metamodel.comparison import *
from mira.metamodel.template_model import *
from mira.metamodel.templates import *
from mira.sources import biomodels

from itertools import combinations
from copy import deepcopy
from tabulate import tabulate

import pandas as pd 

## Define the Covid epi models to be compared and add them to a list 

In [ ]:
COVID_MODEL_LIST = ["BIOMD0000000955", "BIOMD0000000956", "BIOMD0000000957","BIOMD0000000958","BIOMD0000000960","BIOMD0000000962"]

rc = get_dkg_refinement_closure()

tm_covid_grounding_list = []
for covid_model in COVID_MODEL_LIST:
    tm_covid_grounding_list.append(biomodels.get_template_model(covid_model))

model_id_name_mapping = {id:tm.annotations.name.split(' ')[0] for id,tm in enumerate(tm_covid_grounding_list)}


## Compute pairwise similarity scores between each grounded model

In [ ]:
tm_covid_comparison = TemplateModelComparison(tm_covid_grounding_list,refinement_func=rc.is_ontological_child)
grounded_scores = tm_covid_comparison.model_comparison.get_similarity_scores()

grounded_scores_df_list = [{'Model1':model_id_name_mapping[d['models'][0]], 'Model2':model_id_name_mapping[d['models'][1]], 'Similarity Score':d['score']} for d in grounded_scores]
df_grounded = pd.DataFrame(grounded_scores_df_list)
df_grounded

## Print the names for the selected grounded covid biomodels to know which ones we are comparing

We select the third (BIOMD0000000957) and sixth (BIOMD0000000962) grounded covid models 

In [ ]:
print(tm_covid_comparison.template_models[2].annotations.name)
print(tm_covid_comparison.template_models[5].annotations.name)

## Create an image comparing the selected grounded models

In [ ]:
TemplateModelDelta.for_jupyter(tm_covid_comparison.template_models[2],tm_covid_comparison.template_models[5],
                                               rc.is_ontological_child, args="-Grankdir=TB")

## Create a list of covid biomodels without grounding 

In [ ]:
tm_covid_no_grounding_list = [] 
for tm in tm_covid_grounding_list:
    copied_tm = deepcopy(tm)
    for template in copied_tm.templates:
        for concept in template.get_concepts():
            concept.identifiers = {}
            concept.context = {}
    tm_covid_no_grounding_list.append(copied_tm)

## Compute pairwise similarity scores between each ungrounded model

We compare grounded and ungrounded model similarity scores side-by-side

In [ ]:
tm_covid_comparison_copy = TemplateModelComparison(tm_covid_no_grounding_list,refinement_func=rc.is_ontological_child)
ungrounded_scores = tm_covid_comparison_copy.model_comparison.get_similarity_scores()

list_of_both = []

for grounded_score,ungrounded_score in zip(grounded_scores,ungrounded_scores):
    list_of_both.append({'Model1':model_id_name_mapping[grounded_score['models'][0]],
                         'Model2':model_id_name_mapping[grounded_score['models'][1]],
                         'sim_dkg':grounded_score['score'],
                        'sim_no_dkg':ungrounded_score['score']})
                    

no_ground_df = pd.DataFrame(list_of_both)
no_ground_df

## Create an image comparing selected ungrounded models

In [ ]:
TemplateModelDelta.for_jupyter(tm_covid_comparison_copy.template_models[2],tm_covid_comparison_copy.template_models[5],
                                               rc.is_ontological_child,args="-Grankdir=TB")

## Reasoning on difference between grounded and ungrounded models

In [ ]:
print(f"The similarity score between models 2 and 5 when grounded is {grounded_scores[11]['score']}. The similarity score for the models when ungrounded is {list_of_both[11]['sim_no_dkg']}. The reasoning for this is because the refinements dissapear and the confirmed concept node is no longer the same between the two models")

In [ ]:
diff_count = 0 
for model_comparison in list_of_both:
    if model_comparison['sim_dkg'] != model_comparison['sim_no_dkg']:
        diff_count += 1
print(f"Out of {len(list_of_both)} pairs of models, {diff_count} pairs of models have different similarity scores when using the Domain Knowledge Graph")